In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
import matplotlib.pyplot as plt
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))
import string

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import log_loss
from sklearn.metrics import zero_one_loss
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score

from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

pd.set_option('display.max_colwidth',100)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
df = pd.read_csv('/content/drive/My Drive/MCM_News_Aggregator/uci-news-aggregator.csv',usecols=['TITLE', 'CATEGORY','PUBLISHER'])
df.head(10)

,TITLE,PUBLISHER,CATEGORY
0,"Fed official says weak data caused by weather, should not slow taper",Los Angeles Times,b
1,Fed's Charles Plosser sees high bar for change in pace of tapering,Livemint,b
2,US open: Stocks fall after Fed official hints at accelerated tapering,IFA Magazine,b
3,"Fed risks falling 'behind the curve', Charles Plosser says",IFA Magazine,b
4,Fed's Plosser: Nasty Weather Has Curbed Job Growth,Moneynews,b
5,Plosser: Fed May Have to Accelerate Tapering Pace,NASDAQ,b
6,Fed's Plosser: Taper pace may be too slow,MarketWatch,b
7,Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014,FXstreet.com,b
8,US jobs growth last month hit by weather:Fed President Charles Plosser,Economic Times,b
9,ECB unlikely to end sterilisation of SMP purchases - traders,Interactive Investor,b


In [4]:
from keras.utils.np_utils import to_categorical

#Converting categories as balanced
num_of_categories = 45639
shuffled = df.reindex(np.random.permutation(df.index))
e = shuffled[shuffled['CATEGORY'] == 'e'][:num_of_categories]
b = shuffled[shuffled['CATEGORY'] == 'b'][:num_of_categories]
t = shuffled[shuffled['CATEGORY'] == 't'][:num_of_categories]
m = shuffled[shuffled['CATEGORY'] == 'm'][:num_of_categories]
concated = pd.concat([e,b,t,m], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index))
concated['LABEL'] = 0

concated.loc[concated['CATEGORY'] == 'e', 'LABEL'] = 0
concated.loc[concated['CATEGORY'] == 'b', 'LABEL'] = 1
concated.loc[concated['CATEGORY'] == 't', 'LABEL'] = 2
concated.loc[concated['CATEGORY'] == 'm', 'LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'CATEGORY' in concated.keys():
    concated.drop(['CATEGORY'], axis=1)

130807    2
4388      0
146835    3
149705    3
132656    2
86696     1
176318    3
28366     0
27123     0
116193    2
Name: LABEL, dtype: int64
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]]


In [5]:
concated.CATEGORY.value_counts()

t    45639
b    45639
m    45639
e    45639
Name: CATEGORY, dtype: int64

In [6]:
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 130807 to 103554
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   TITLE      182556 non-null  object
 1   PUBLISHER  182555 non-null  object
 2   CATEGORY   182556 non-null  object
 3   LABEL      182556 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 7.0+ MB


In [7]:
# Concating title and publisher column
concated['title_publish'] = concated.TITLE + " " + concated.PUBLISHER
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182556 entries, 130807 to 103554
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TITLE          182556 non-null  object
 1   PUBLISHER      182555 non-null  object
 2   CATEGORY       182556 non-null  object
 3   LABEL          182556 non-null  int64 
 4   title_publish  182555 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.4+ MB


In [8]:
concated.CATEGORY.value_counts()

t    45639
b    45639
m    45639
e    45639
Name: CATEGORY, dtype: int64

In [9]:
# Drop null
concated= concated.dropna(subset=['title_publish'])

In [10]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish
130807,"Nest Learning Thermostat goes on sale, hopes to save households cash",Digital Spy UK,t,2,"Nest Learning Thermostat goes on sale, hopes to save households cash Digital Spy UK"
4388,Record Store Day is music to fans' ears,Inland Valley Daily Bulletin,e,0,Record Store Day is music to fans' ears Inland Valley Daily Bulletin
146835,"Extreme Obesity Kills: As Number Of Fat Americans Soars, Scientists Confirm ...",Medical Daily,m,3,"Extreme Obesity Kills: As Number Of Fat Americans Soars, Scientists Confirm ... Medical Daily"
149705,Man loses nearly 400 pounds after meeting inspiring friend online,WGNtv.com,m,3,Man loses nearly 400 pounds after meeting inspiring friend online WGNtv.com
132656,Only 'blind luck' has saved us from big asteroids so far: says ex- NASA astronaut,French Tribune,t,2,Only 'blind luck' has saved us from big asteroids so far: says ex- NASA astronaut French Tribune


In [11]:
symbols = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', 
           ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', 
           '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', 
           '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', 
           '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', 
           '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', 
           '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', 
           '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', 
           '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', 
           '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', 
           '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', 
           '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', 
           '¹', '≤', '‡', '√', ]

def clean_symbol(text):
    text = str(text)
    for symbol in symbols:
        text = text.replace(symbol, ' ')
    return text

# remove symbols and punctuations 
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_symbol(x))

In [12]:
# Dictionary of short form words and mispellings
short_forms_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                    "'cause": "because", "could've": "could have", "couldn't": "could not", 
                    "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", 
                    "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                    "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                    "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", 
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                    "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                    "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                    "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", 
                    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
                    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", 
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                    "she'll've": "she will have", "she's": "she is", "should've": "should have", 
                    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", 
                    "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                    "there's": "there is", "here's": "here is","they'd": "they would", 
                    "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", 
                    "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", 
                    "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                    "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", 
                    "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", 
                    "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have",
                    "y'all're": "you all are","y'all've": "you all have","you'd": "you would", 
                    "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", 
                    "you're": "you are", "you've": "you have"}

def clean_shortforms(text):
    clean_text = text
    for shortform in short_forms_dict.keys():
        if re.search(shortform, text):
            clean_text = re.sub(shortform, short_forms_dict[shortform], text)
    return clean_text

# fix short forms
concated['title_publish'] = concated['title_publish'].apply(lambda x: clean_shortforms(x))

In [13]:
concated['title_publish']  = concated['title_publish'] .str.replace('[^\w\s]',' ')# unpunctuate

In [14]:
import re
import string

def clean_text(text):
    #text = re.sub("[0-9]+"," ",text)
    text = text.lower() # lowercase text
    text = re.sub(' +',' ', text)
    tokens=re.split('\W+',text)
    text1 = " ".join([word for word in tokens if word not in stop_words and word not in string.punctuation])
    return text1

In [15]:
concated['clean_title_publish'] = concated['title_publish'].apply(lambda x:clean_text(x))

In [16]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
130807,"Nest Learning Thermostat goes on sale, hopes to save households cash",Digital Spy UK,t,2,Nest Learning Thermostat goes on sale hopes to save households cash Digital Spy UK,nest learning thermostat goes sale hopes save households cash digital spy uk
4388,Record Store Day is music to fans' ears,Inland Valley Daily Bulletin,e,0,Record Store Day is music to fans ears Inland Valley Daily Bulletin,record store day music fans ears inland valley daily bulletin
146835,"Extreme Obesity Kills: As Number Of Fat Americans Soars, Scientists Confirm ...",Medical Daily,m,3,Extreme Obesity Kills As Number Of Fat Americans Soars Scientists Confirm Medical Daily,extreme obesity kills number fat americans soars scientists confirm medical daily
149705,Man loses nearly 400 pounds after meeting inspiring friend online,WGNtv.com,m,3,Man loses nearly 400 pounds after meeting inspiring friend online WGNtv com,man loses nearly 400 pounds meeting inspiring friend online wgntv com
132656,Only 'blind luck' has saved us from big asteroids so far: says ex- NASA astronaut,French Tribune,t,2,Only blind luck has saved us from big asteroids so far says ex NASA astronaut French Tribune,blind luck saved us big asteroids far says ex nasa astronaut french tribune


In [17]:
import re
import string

def clean_text1(text):
    text = string.capwords(text, sep = None) 
    tokens=re.split('\W+',text)
    tokens = list(set(tokens))
    text1 = [lemmatizer.lemmatize(word) for word in tokens if (len(word) > 2 and len(word) < 14)]
    return text1

In [18]:
concated['clean_title_publish'] = concated['clean_title_publish'].apply(lambda x:clean_text1(x))

In [19]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
130807,"Nest Learning Thermostat goes on sale, hopes to save households cash",Digital Spy UK,t,2,Nest Learning Thermostat goes on sale hopes to save households cash Digital Spy UK,"[Households, Thermostat, Cash, Sale, Learning, Hopes, Spy, Nest, Digital, Save, Goes]"
4388,Record Store Day is music to fans' ears,Inland Valley Daily Bulletin,e,0,Record Store Day is music to fans ears Inland Valley Daily Bulletin,"[Inland, Fans, Bulletin, Ears, Day, Music, Store, Daily, Record, Valley]"
146835,"Extreme Obesity Kills: As Number Of Fat Americans Soars, Scientists Confirm ...",Medical Daily,m,3,Extreme Obesity Kills As Number Of Fat Americans Soars Scientists Confirm Medical Daily,"[Scientists, Medical, Extreme, Soars, Number, Kills, Daily, Americans, Confirm, Obesity, Fat]"
149705,Man loses nearly 400 pounds after meeting inspiring friend online,WGNtv.com,m,3,Man loses nearly 400 pounds after meeting inspiring friend online WGNtv com,"[Online, Nearly, Inspiring, Man, Wgntv, Com, Loses, 400, Pounds, Friend, Meeting]"
132656,Only 'blind luck' has saved us from big asteroids so far: says ex- NASA astronaut,French Tribune,t,2,Only blind luck has saved us from big asteroids so far says ex NASA astronaut French Tribune,"[Astronaut, Far, Blind, French, Saved, Big, Luck, Asteroids, Nasa, Says, Tribune]"


In [20]:
concated.tail(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish
181883,Many Ivy League Students Take Study Drugs And Don't Consider It Cheating,Huffington Post,m,3,Many Ivy League Students Take Study Drugs And Don t Consider It Cheating Huffington Post,"[Huffington, Cheating, Study, Take, Drugs, Many, League, Ivy, Consider, Students, Post]"
78181,Russia diplomat says US high-tech export curbs will be a blow,Reuters,b,1,Russia diplomat says US high tech export curbs will be a blow Reuters,"[Diplomat, Russia, Export, Tech, Curbs, Says, Blow, Reuters, High]"
74132,Alibaba just revealed how much money it makes,Quartz,b,1,Alibaba just revealed how much money it makes Quartz,"[Makes, Alibaba, Money, Quartz, Revealed, Much]"
51222,July Fourth sparks high gas prices,KSDK,b,1,July Fourth sparks high gas prices KSDK,"[Prices, July, Ksdk, Fourth, Sparks, Gas, High]"
103554,Brand new Galaxy S5 outclassed by HTC One (M8) and six-month-old iPhone 5s ...,BGR,t,2,Brand new Galaxy S5 outclassed by HTC One M8 and six month old iPhone 5s BGR,"[Iphone, Bgr, Outclassed, New, One, Month, Htc, Brand, Galaxy, Old, Six]"


In [21]:
import re
import string

def clean_text2(text):
    text1 = " ".join([lemmatizer.lemmatize(word) for word in text if (len(word) > 2 and len(word) < 14)])
    return text1

In [22]:
concated['clean_title_publish1'] = concated['clean_title_publish'].apply(lambda x:clean_text2(x))

In [23]:
concated.head(5)

,TITLE,PUBLISHER,CATEGORY,LABEL,title_publish,clean_title_publish,clean_title_publish1
130807,"Nest Learning Thermostat goes on sale, hopes to save households cash",Digital Spy UK,t,2,Nest Learning Thermostat goes on sale hopes to save households cash Digital Spy UK,"[Households, Thermostat, Cash, Sale, Learning, Hopes, Spy, Nest, Digital, Save, Goes]",Households Thermostat Cash Sale Learning Hopes Spy Nest Digital Save Goes
4388,Record Store Day is music to fans' ears,Inland Valley Daily Bulletin,e,0,Record Store Day is music to fans ears Inland Valley Daily Bulletin,"[Inland, Fans, Bulletin, Ears, Day, Music, Store, Daily, Record, Valley]",Inland Fans Bulletin Ears Day Music Store Daily Record Valley
146835,"Extreme Obesity Kills: As Number Of Fat Americans Soars, Scientists Confirm ...",Medical Daily,m,3,Extreme Obesity Kills As Number Of Fat Americans Soars Scientists Confirm Medical Daily,"[Scientists, Medical, Extreme, Soars, Number, Kills, Daily, Americans, Confirm, Obesity, Fat]",Scientists Medical Extreme Soars Number Kills Daily Americans Confirm Obesity Fat
149705,Man loses nearly 400 pounds after meeting inspiring friend online,WGNtv.com,m,3,Man loses nearly 400 pounds after meeting inspiring friend online WGNtv com,"[Online, Nearly, Inspiring, Man, Wgntv, Com, Loses, 400, Pounds, Friend, Meeting]",Online Nearly Inspiring Man Wgntv Com Loses 400 Pounds Friend Meeting
132656,Only 'blind luck' has saved us from big asteroids so far: says ex- NASA astronaut,French Tribune,t,2,Only blind luck has saved us from big asteroids so far says ex NASA astronaut French Tribune,"[Astronaut, Far, Blind, French, Saved, Big, Luck, Asteroids, Nasa, Says, Tribune]",Astronaut Far Blind French Saved Big Luck Asteroids Nasa Says Tribune


In [24]:
concated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182555 entries, 130807 to 103554
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   TITLE                 182555 non-null  object
 1   PUBLISHER             182555 non-null  object
 2   CATEGORY              182555 non-null  object
 3   LABEL                 182555 non-null  int64 
 4   title_publish         182555 non-null  object
 5   clean_title_publish   182555 non-null  object
 6   clean_title_publish1  182555 non-null  object
dtypes: int64(1), object(6)
memory usage: 11.1+ MB


In [25]:
%%time
from gensim.models import FastText

#Dimension of vectors we are generating
EMBEDDING_DIM = 300
wv = FastText(sentences=concated['clean_title_publish'], size=EMBEDDING_DIM, window=5, min_count=1)  

CPU times: user 3min 27s, sys: 1.44 s, total: 3min 28s
Wall time: 1min 55s


In [26]:
#printing similarity index
print(wv.most_similar('English'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('Enetenglish', 0.9406661987304688), ('Crienglish', 0.9035052061080933), ('Echos', 0.9004624485969543), ('Jailpublish', 0.873024046421051), ('Establish', 0.8505996465682983), ('Gulf', 0.843049168586731), ('Gulps', 0.8427060842514038), ('Anguish', 0.8377653360366821), ('Anglia', 0.8368692994117737), ('Epoch', 0.8298892974853516)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [27]:
wv['English']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.49844074,  0.23064356, -0.07958514, -0.26164666,  0.01949881,
        0.58104414, -0.21107244,  0.05993776, -0.16126335, -0.1211302 ,
        0.08544911, -0.07795384,  0.36289197,  0.3356716 , -0.1352066 ,
        0.15837267,  0.13490167,  0.07117746,  0.16834496,  0.04412734,
        0.09810511, -0.20331278, -0.34036854, -0.05774774,  0.5565769 ,
        0.11725187, -0.15077433,  0.43581033,  0.45329648,  0.022492  ,
       -0.06135816, -0.10060461,  0.2709122 ,  0.13974595, -0.09307916,
        0.47928038,  0.12736428,  0.13224606,  0.66474676, -0.6246111 ,
        0.08253914, -0.37692198, -0.02068034, -0.02522929, -0.00593534,
        0.50379723,  0.31066355, -0.15962452, -0.2579443 ,  0.27419668,
       -0.3337049 , -0.18875778, -0.6263115 , -0.19649926,  0.0306331 ,
       -0.10855799, -0.32771397, -0.21138595, -0.18578772, -0.04067463,
       -0.19321212,  0.266695  , -0.29072437, -0.13175204, -0.25374827,
        0.11095784,  0.11220452,  0.06677659,  0.24722947,  0.27

In [28]:
#vocab size
len(wv.wv.vocab)
#We have now represented each words by a 300dim vector.

42410

In [29]:
#Word Averaging
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)


    if not mean:
        #logging.warning("cannot compute similarity with no input %s", words)
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, clean_title_publish) for clean_title_publish in text_list])

In [30]:
# Tokenization
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [31]:
%%time
train, test = train_test_split(concated, test_size=0.3,shuffle=True)
test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['clean_title_publish1']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['clean_title_publish1']), axis=1).values

CPU times: user 38.9 s, sys: 333 ms, total: 39.3 s
Wall time: 39.5 s


In [32]:
%%time
X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


CPU times: user 13.9 s, sys: 190 ms, total: 14.1 s
Wall time: 14.1 s


Log Regression

In [33]:
"""
# Hyper parameter tuning
# multi_class
multi_class = ['multinomial']

# solver
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
 
# class_weight
class_weight = ['balanced', None]

# penalty
penalty = ['l2']

# Create the random grid
random_grid = {
               'multi_class': multi_class,
               'solver': solver,
               'class_weight': class_weight,
               'penalty': penalty}

"""

"\n# Hyper parameter tuning\n# multi_class\nmulti_class = ['multinomial']\n\n# solver\nsolver = ['newton-cg', 'sag', 'saga', 'lbfgs']\n \n# class_weight\nclass_weight = ['balanced', None]\n\n# penalty\npenalty = ['l2']\n\n# Create the random grid\nrandom_grid = {\n               'multi_class': multi_class,\n               'solver': solver,\n               'class_weight': class_weight,\n               'penalty': penalty}\n\n"

In [34]:
"""

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# First create the base model to tune
lrc = LogisticRegression(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=lrc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='accuracy',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(X_train_word_average, train['LABEL'])

"""

"\n\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.model_selection import GridSearchCV\n# First create the base model to tune\nlrc = LogisticRegression(random_state=8)\n\n# Definition of the random search\nrandom_search = RandomizedSearchCV(estimator=lrc,\n                                   param_distributions=random_grid,\n                                   n_iter=50,\n                                   scoring='accuracy',\n                                   cv=3, \n                                   verbose=1, \n                                   random_state=8)\n\n# Fit the random search model\nrandom_search.fit(X_train_word_average, train['LABEL'])\n\n"

In [35]:
"""
print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

"""


'\nprint("The best hyperparameters from Random Search are:")\nprint(random_search.best_params_)\nprint("")\nprint("The mean accuracy of a model with these hyperparameters is:")\nprint(random_search.best_score_)\n\n'

In [36]:
# First create the base model to tune
logreg = LogisticRegression(n_jobs=-1,solver='sag',penalty='l2',multi_class='multinomial',class_weight='balanced')

# Fit the random search model
logreg.fit(X_train_word_average, train['LABEL'])

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='multinomial', n_jobs=-1,
                   penalty='l2', random_state=None, solver='sag', tol=0.0001,
                   verbose=0, warm_start=False)

In [37]:
y_pred_test_log=logreg.predict(X_test_word_average)
y_pred_train_log=logreg.predict(X_train_word_average)

In [38]:
d=accuracy_score(y_pred_test_log,test.LABEL)*100
print ('Test Accuracy',d)

e=accuracy_score(y_pred_train_log,train.LABEL)*100
print ('Train Accuracy',e)

Test Accuracy 82.81812040097138
Train Accuracy 82.75424922527937


In [39]:
bac=balanced_accuracy_score(y_pred_test_log,test.LABEL)*100
c_k_s=cohen_kappa_score(y_pred_test_log,test.LABEL)*100
mc=matthews_corrcoef(y_pred_test_log,test.LABEL)*100


print(c_k_s)
print(mc)
print(bac)

77.09091753386978
77.10174697259315
82.80537709226373


SGD

In [40]:
%%time
sgd_search = SGDClassifier(n_jobs=-1, max_iter=20,alpha=0.00001)
sgd_search.fit(X_train_word_average,  train['LABEL'])

y_pred_test_sgd = sgd_search.predict(X_test_word_average)
y_pred_train_sgd = sgd_search.predict(X_train_word_average)


CPU times: user 12.8 s, sys: 210 ms, total: 13 s
Wall time: 6.89 s


In [41]:
f=accuracy_score(y_pred_test_sgd,test.LABEL)*100
print ('Test Accuracy',f)

g=accuracy_score(y_pred_train_sgd,train.LABEL)*100
print ('Train Accuracy',g)

Test Accuracy 82.7523873865649
Train Accuracy 82.69164553792218


In [42]:
print(classification_report(test.LABEL, y_pred_test_sgd))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87     13655
           1       0.78      0.81      0.80     13703
           2       0.84      0.75      0.79     13688
           3       0.83      0.86      0.85     13721

    accuracy                           0.83     54767
   macro avg       0.83      0.83      0.83     54767
weighted avg       0.83      0.83      0.83     54767



In [43]:
# title-category function
def title_to_category1(title):
    title=clean_text(title)
    title=clean_text1(title)
    title=clean_symbol(title)
    title=clean_shortforms(title)
    categories = {1 : 'business', 
                  2 : 'science and technology', 
                  0 : 'entertainment', 
                  3 : 'health'}
    pridicter1 = logreg.predict(word_averaging_list(wv,w2v_tokenize_text(title)))
    return categories[pridicter1[0]]

In [44]:
title_to_category1('12 family getaways in Ireland this summer')

'science and technology'

In [45]:
# title-category function
def title_to_category2(title):
    title=clean_text(title)
    title=clean_text1(title)
    title=clean_symbol(title)
    title=clean_shortforms(title)
    categories = {1 : 'business', 
                  2 : 'science and technology', 
                  0 : 'entertainment', 
                  3 : 'health'}
    pridicter2 = sgd_search.predict(word_averaging_list(wv,w2v_tokenize_text(title)))
    return categories[pridicter2[0]]

In [46]:
title_to_category2('Whole four-star hotel up for grabs in charity raffle')

'science and technology'